### Etymology

In [ ]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse

In [ ]:
response = requests.get("https://etymologeek.com/search/all/algebra")
response

In [ ]:
soup = BeautifulSoup(response.content, 'lxml')

In [ ]:
word = soup.find('tbody', attrs={"id":"tb"})
word

In [ ]:
dict_word = word.findAll('td', attrs={"data-th":"Dic. entry"})
dict_word

In [ ]:
lang = word.findAll('td', attrs={"data-th":"Language"})
lang

In [ ]:
definition = word.findAll('td', attrs={"data-th":"Definition"})
definition

In [ ]:
a_href = word.findAll('td', attrs={"data-th":"Dic. entry"})
a_href

In [ ]:
iter_values = zip(dict_word, lang, definition, a_href)

In [ ]:
result_list = []
for i, j, k, m in iter_values:
    var1 = i.string
    var2 = j.string
    var3 = k.string
    var4 = m.find('a').get('href')
    result_list.append([var1,var2,var3, var4])

In [ ]:
pd.DataFrame(result_list)

#### For Loop

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
df_word = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/English/Result/Word/Merge/Word_Merge2.xlsx")
df_word = df_word.head(10000)
df_word

In [ ]:
word_lemma = []
for word in df_word["word"]:
    word_lemma.append(WordNetLemmatizer().lemmatize(word))
    word_unique = set(word_lemma)

In [ ]:
len(word_lemma)

In [ ]:
len(word_unique)

In [ ]:
result_list = []
for i in word_unique:
#for i in word_list:
    try:
        response = requests.get(f"https://etymologeek.com/search/all/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_word = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        a_href = word.findAll('td', attrs={"data-th":"Dic. entry"})
        iter_values = zip(dict_word, lang, definition, a_href)
        for a, b, c, d in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()  # string
            var3 = c.get_text()  # string
            var4 = d.find('a').get('href')
            var5 = f"https://etymologeek.com{var4}"
            result_list.append([i, var1, var2, var3, var5])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"dict_entry", 2:"language", 3:"definition", 4:"href"}, inplace=True)
df_result

In [ ]:
df_result.to_excel("English_Etymologeek_Main_Result.xlsx", index=False)

In [ ]:
df_main_word = pd.read_excel("English_Etymologeek_Main_Result.xlsx")
df_main_word

In [ ]:
df_main_word.search_word.nunique()

In [ ]:
df_main_word.language.nunique()

In [ ]:
search_word = df_main_word["search_word"].to_list()
dict_entry_main = df_main_word["dict_entry"].to_list()
language_main = df_main_word["language"].to_list()
definition_main = df_main_word["definition"].to_list()	
href_url = df_main_word["href"].to_list()
main_iter_values = zip(search_word, dict_entry_main, language_main, definition_main, href_url)
result_list2 = []
for search, dict_ent, lang_main, def_main, href in main_iter_values:
    try:
        response = requests.get(f"{href}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_entry = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        iter_values = zip(dict_entry, lang, definition)
        for a, b, c in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()
            var3 = c.get_text()
            result_list2.append([search, dict_ent, lang_main, def_main, var1, var2, var3])
    except:
        pass

df_result2 = pd.DataFrame(result_list2)
df_result2.rename(columns={0:"search_word", 1:"dict_entry_main", 2:"language_main", 3:"definition_main", 4:"dict_entry", 5:"language", 6:"definition"}, inplace=True)
df_result2

In [ ]:
df_result2.language_main = df_result2.language_main.apply(lambda x: x.strip())
df_result2.language = df_result2.language.apply(lambda x: x.strip())

In [ ]:
df_result2.language_main.unique()

In [ ]:
df_result2.language.unique()

In [ ]:
df_result2.to_excel("English_Etymologeek_Main_And_Additional_Result.xlsx", index=False)

In [ ]:
df_lang_pair = df_result2[(df_result2["language_main"] == "English") & (df_result2["language"] == "French (fra)")]
df_lang_pair

In [ ]:
df_lang_pair.to_excel("English_And_French_Etymologeek_Result.xlsx", index=False)